In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import os
import s3fs

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.regularizers import l2

from modules.utils import pre_process_data

2024-11-30 09:38:58.644344: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 09:38:58.650346: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 09:38:58.666448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732959538.692313  158236 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732959538.698685  158236 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 09:38:58.722631: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "ebahri-ensae"
FILE_KEY_S3 = "X_train_Hi5.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    x_train = pd.read_csv(file_in, sep=",")

/tmp/ipykernel_158236/992204584.py:8: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  x_train = pd.read_csv(file_in, sep=",")


In [3]:
x_trained_without_nan, y_train = pre_process_data(x_train)

/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, ou

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, ou

In [5]:
y_train = pd.DataFrame(y_train)
y_train.head()

,0
0,1
1,3
2,1
3,3
4,4


In [6]:
x_trained_without_nan.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_station_bss_id,...,prelev_volume_2,prelev_usage_label_2,prelev_volume_obtention_mode_label_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,region
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,5.748241,45.828148,BSS001TTQQ,...,1.038098e+10,EAU TURBINEE (barrage),Mesure indirecte,1.793055e+09,11.8,992.0,25250,2.9,16.2,Auvergne-Rhône-Alpes
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,5.356637,46.028102,BSS001SCTM,...,1.818150e+09,EAU TURBINEE (barrage),Mesure indirecte,1.085125e+09,0.6,1786.0,24660,44.5,11,Auvergne-Rhône-Alpes
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,5.220795,45.895734,BSS001TRPH,...,6.981240e+08,EAU TURBINEE (barrage),Mesure indirecte,3.810492e+08,0,8085.0,24890,8.4,7.8,Auvergne-Rhône-Alpes
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,5.948977,46.201180,BSS001RGXM,...,1.843691e+08,EAU TURBINEE (barrage),Mesure indirecte,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Auvergne-Rhône-Alpes
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,5.313353,46.136402,BSS001RFRV,...,9.263080e+08,EAU TURBINEE (barrage),Mesure indirecte,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Auvergne-Rhône-Alpes


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_trained_without_nan,y_train ))

In [ ]:
# Calculate sizes for train and validation sets
dataset_size = len(x_trained_without_nan)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 16
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
model = Sequential([
    # First Dense Layer: Input layer with more neurons
    Dense(512, activation='relu', input_shape=(80,)),  # Increased neurons to 512
    BatchNormalization(),
    Dropout(0.2),

    # Second Dense Layer: Increased neurons and added layer
    Dense(256, activation='relu'),  # Increased neurons to 256
    BatchNormalization(),
    Dropout(0.3),

    # Third Dense Layer: Further increased neurons
    Dense(128, activation='relu'),  # Increased neurons to 128
    BatchNormalization(),
    Dropout(0.4),

    # Fourth Dense Layer: Further refinement
    Dense(64, activation='relu'),  # Increased neurons to 64
    BatchNormalization(),
    Dropout(0.5),

    # Fifth Dense Layer: Further refinement with reduced neurons
    Dense(32, activation='relu'),  # Reduced neurons to 32
    BatchNormalization(),
    Dropout(0.5),

    # Sixth Dense Layer: Even further reduction for more abstract features
    Dense(16, activation='relu'),  # Reduced neurons to 16
    BatchNormalization(),
    Dropout(0.5),

    # Seventh Dense Layer: Final layer for deep abstraction
    Dense(8, activation='relu'),  # Reduced neurons to 8
    BatchNormalization(),
    Dropout(0.5),

    # Output Layer: Softmax activation for classification
    Dense(3, activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-28          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/third_model_8.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)


history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=500,
    callbacks=[reduce_lr, checkpoint]
)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.grid(True)